# Exploration and a Simple Baseline

In this part of the assignment, we'll introduce the [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/index.html) (SST) dataset, and train a Naive Bayes model as a simple baseline. The SST, introduced by [(Socher et al. 2013)](http://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf) consists of approximately 10,000 sentences from movie reviews associated with fine-grained sentiment labels on a five-point scale, and is a popular benchmark for text classification.

## Outline

- **Part (a):** The Stanford Sentiment Treebank
- **Part (b):** Naive Bayes
- **Part (c):** Exploring Negation

Exercises are interspersed throughout the notebook. Be sure you catch all of them! There are 4 questions for Part (a), 2 for Part (b), and 3 for Part (c).

In [1]:
# Install a few python packages using pip
from w266_common import utils
utils.require_package("wget")      # for fetching dataset
utils.require_package("bokeh")     # for plotting histograms
utils.require_package("graphviz")  # for rendering trees

### Preliminaries: GraphViz

This notebook uses [GraphViz](https://www.graphviz.org/) to render tree structures. We'll use this several times later in the course, particularly on Assignment 5, so it's worth having. On Ubuntu / Debian (including Google Cloud), you can install it by running on the command line:
```
sudo apt-get install graphviz
```

For Mac OSX, you can install using Homebrew:
```
brew install graphviz
```
or see https://www.graphviz.org/download/ for more options. Run the cell below to set up rendering and show a sample tree.

In [2]:
import nltk
from w266_common import treeviz
import sst
# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

# Test rendering
print("Sample tree to test rendering:")
nltk.tree.Tree.fromstring("(4 (4 (2 I) (3 love) (3 W266)) (3 😄))")

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.
Sample tree to test rendering:


Tree('4', [Tree('4', [Tree('2', ['I']), Tree('3', ['love']), Tree('3', ['W266'])]), Tree('3', ['😄'])])

In [3]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK, NumPy, and Pandas.
import nltk
import numpy as np
import pandas as pd

# Helper libraries.
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst, models, models_test

# Bokeh for plotting.
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

# Helper code for plotting histograms
def plot_length_histogram(lengths, x_range=[0,100], bins=40, normed=True):
    hist, bin_edges = np.histogram(a=lengths, bins=bins, normed=normed, range=x_range)
    bin_centers = (bin_edges[1:] + bin_edges[:-1])/2
    bin_widths =  (bin_edges[1:] - bin_edges[:-1])

    hover = HoverTool(tooltips=[("bucket", "@x"), ("count", "@top")], mode="vline")
    fig = bp.figure(plot_width=800, plot_height=400, tools=[hover])
    fig.vbar(x=bin_centers, width=bin_widths, top=hist, hover_fill_color="firebrick")
    fig.y_range.start = 0
    fig.x_range.start = 0
    fig.xaxis.axis_label = "Example length (number of tokens)"
    fig.yaxis.axis_label = "Frequency"
    bp.show(fig)

/home/yeunghoman/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Loading BokehJS ...

# Part (a): The Stanford Sentiment Treebank

_If you haven't yet, be sure to familiarize yourself with the [Prelude notebook](Prelude.ipynb), as this assignment will assume familiarity with the text pre-processing steps described there._

The [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/index.html) (SST) is one of the most widely used datasets as a benchmark for text classification. It consists of 11,855 sentences drawn from a corpus of movie reviews (originally from Rotten Tomatoes), each labeled with sentiment on a five-point scale.

For example:
```
sentence: [A warm , funny , engaging film .]
label:    4 (very positive)
```

For this assignment, we'll work with the binarized form of the dataset, where the lowest two classes are mapped to a single "negative" label, the highest two are mapped to a single "positive" label, and neutral examples are omitted.

**Side note:**
Unlike most classification datasets, SST is also a _treebank_, which means each sentence is associated with a tree structure that decomposes it into subphrases. So for the example above, we'd also have sentiment labels for `[warm , funny]` and `[engaging film .]` and so on. The trees are created by running the [Stanford Parser](https://nlp.stanford.edu/software/lex-parser.shtml) over the original sentences, then crowdsourcing sentiment labels on each sub-phrase. We'll talk more about treebanks later in the course, and in Assignment 5 you'll implement a simple version of the Stanford Parser.

For the purposes of this assignment, we'll mostly concern ourselves with the sentence-level ("root") labels, but the tree structure will come in handy in two places:
- As a way of analyzing the examples to find instances of negation
- (optionally) As a source of additional training data, by including phrase labels

### Obtaining the Data
The data is distributed as serialized trees in [S-expression](https://en.wikipedia.org/wiki/S-expression) form, like this:
```
(4 (4 (2 A) (4 (3 (3 warm) (2 ,)) (3 funny))) (3 (2 ,) (3 (4 (4 engaging) (2 film)) (2 .))))
```

We've provided an `SSTDataset` class (in `sst.py`) which will download the dataset and parse the S-expressions into [`nltk.tree.Tree`](http://www.nltk.org/api/nltk.html?highlight=tree#nltk.tree.Tree) objects that you can easily view in the notebook.

`SSTDataset` also implements the text-processing pipeline described in the [Prelude notebook](Prelude.ipynb), and provides methods (`as_sparse_bow` and `as_padded_array`) to convert the data to matrix form.

Run the cell below; it will download a ~6MB .zip file to the local directory the first time you run it.

In [4]:
import sst
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")

Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


A few members of the `SSTDataset()` class that you might find useful:
- **`ds.vocab`**: a `vocabulary.Vocabulary` object managing the model vocabulary
- **`ds.{train,dev,test}_trees`**: a list of [`nltk.tree.Tree`](http://www.nltk.org/api/nltk.html?highlight=tree#nltk.tree.Tree) objects representing each sentence
- **`ds.{train,dev,test}`**: a Pandas DataFrame containing the _processed_ examples, including all subphrases. `label` is the target label, `is_root` denotes whether this example is a root node (full sentence), and `root_id` is the index of the tree that the example was derived from.

In [5]:
# Look at a tree for a positive review
ds.dev_trees[3]

Tree('4', [Tree('4', [Tree('2', ['A']), Tree('4', [Tree('3', [Tree('3', ['warm']), Tree('2', [','])]), Tree('3', ['funny'])])]), Tree('3', [Tree('2', [',']), Tree('3', [Tree('4', [Tree('4', ['engaging']), Tree('2', ['film'])]), Tree('2', ['.'])])])])

In [6]:
# Look at a tree for a negative review
ds.dev_trees[361]

Tree('0', [Tree('2', ['It']), Tree('2', [Tree('2', [Tree('2', ["'s"]), Tree('0', [Tree('2', ['like']), Tree('1', [Tree('2', ['watching']), Tree('0', [Tree('1', [Tree('2', ['a']), Tree('0', ['nightmare'])]), Tree('2', [Tree('2', ['made']), Tree('2', ['flesh'])])])])])]), Tree('2', ['.'])])])

## Part (a) questions: Exploring the Data

Let's explore the data a bit, to get a sense of what we're working with. If you're not familiar with DataFrames, you may wish to review the Pandas documentation: https://pandas.pydata.org/pandas-docs/stable/dsintro.html 

Answer the following questions in the cells below:

1. Looking at only the root examples in the training set (*Hint: use `ds.train[ds.train.is_root]`*), what is the fraction of positive labels? Is the classification task balanced, or close to it? If we used most-common-class as a baseline classifier, what would the accuracy be?
2. What are the five most common words in the dataset? (*Hint: there are several ways to get at this - you might use `collections.Counter`, or poke around in the `ds.vocab` object.*)
3. Use the `plot_length_histogram` function (defined above) to plot a histogram of the sentence lengths. What is the 95% percentile length? (i.e. 95% of examples in the training set should be shorter than this)
4. Repeat 3., but this time including all subphrases. Notice the difference in distributions. Could this be problematic, if we include subphrases in our training data?

## Answers for Part (a)
<a id="answers_a1234"></a>

1. 0.5216763005780347. The targets are close to balanced. The most common class is label 1. If we assign predict every example to be 1, then we have accuracy 0.5216763005780347
2. Including the punctuation marks : ".", "the", ",", "a", "and". Excluding the punctuation marks: "the", "a", "and", "of", "to". Please see the cell output for counts.
3. 95% of the sentences have less than 36 words.
4. 95% of the sentences have less than 24 words because subphrases are accounted now. The distribution becomes extremely skewed. What happens is that words deeper in the parse trees appear in more sub-phrases for each complete sentence and we over count them a lot here. This can be problematic for our downstream tasks. For instance, a predictive model can be over-senstive to words that tend to be buried deeper in the parse tree but in fact have no sensitiment values.

Use the cells below for your code solutions. Each part shouldn't require more than a couple lines, but you're welcome to explore more!

In [7]:
#### YOUR CODE HERE ####
# Code for Part (a).1
np.mean(ds.train[ds.train.is_root].label)
#### END(YOUR CODE) ####

0.5216763005780347

In [30]:
#### YOUR CODE HERE ####
# Code for Part (a).2
# print top 7 to look beyond punctuation marks
collections.Counter(word for group in ds.train[ds.train.is_root].tokens for word in group).most_common(7)
#### END(YOUR CODE) ####

[('.', 6538),
 ('the', 5954),
 (',', 5883),
 ('a', 4361),
 ('and', 3831),
 ('of', 3631),
 ('to', 2438)]

In [5]:
#### YOUR CODE HERE ####
# Code for Part (a).3
sent_lens = ds.train[ds.train.is_root].tokens.apply(lambda x : len(x))
plot_length_histogram(lengths=sent_lens, x_range=[0,100], bins=100, normed=True)
print ("95% percentile for sentence length = ",int(np.percentile(a=sent_lens, q=95)))
#### END(YOUR CODE) ####

95% percentile for sentence length =  36


In [32]:
#### YOUR CODE HERE ####
# Code for Part (a).4
sent_lens = ds.train.tokens.apply(lambda x : len(x))
plot_length_histogram(lengths=sent_lens, x_range=[0,100], bins=100, normed=True)
print ("95% percentile for sentence length = ",int(np.percentile(a=sent_lens, q=95)))
#### END(YOUR CODE) ####

95% percentile for sentence length =  24


# Part (b): Naive Bayes

In this section, we'll build and explore a Naive Bayes model as a baseline classifier for our dataset.

Naive Bayes is perhaps the simplest possible classification algorithm, but it's one that still surprisingly effective for many text classification problems. Recall from your Machine Learning course:

$$ P(y = k) = \hat{\theta}_k = \frac{1}{N}\sum_{i = 1}^N \mathbf{1}[y_i = k] $$

$$ P(x_j | y = 1) = \hat{\theta}_{k,j} = 
\frac{ 
\sum_{i = 1}^N  \sum_{j' = 1}^{n_i} \mathbf{1}[y_i = 1 \wedge x_{j'} = j]
}{
\sum_{i = 1}^N  \mathbf{1}[y_i = 1] \cdot n_i
}
$$

where $N$ is the size of the dataset, and $n_i$ is the length (number of tokens of the $i^{th}$ example. Prediction is done by computing the score:

$$ \mathrm{score}(x) = \log \left(\frac{P(y = 1) \prod_{j=1}^n P(x_j | y = 1)}{P(y = 0) \prod_{j=1}^n P(x_j | y = 0)}\right) $$

We'll just use the [implementation from scikit-learn](http://scikit-learn.org/stable/modules/naive_bayes.html). Like other scikit-learn classifiers, this expects the input as a `scipy.sparse` matrix. Run the cell below:

In [6]:
# 'csr' stands for "Compressed Sparse Row", which is one format
# for representing sparse matricies.
train_x_csr, train_y = ds.as_sparse_bow("train", root_only=True)
test_x_csr,  test_y  = ds.as_sparse_bow("test", root_only=True)
print("Training set: x = {:s} sparse, y = {:s}".format(str(train_x_csr.shape), 
                                                str(train_y.shape)))
print("Test set:     x = {:s} sparse, y = {:s}".format(str(test_x_csr.shape), 
                                                str(test_y.shape)))

Training set: x = (6920, 16474) sparse, y = (6920,)
Test set:     x = (1821, 16474) sparse, y = (1821,)


Note the `root_only=True` parameter - this will return only examples corresponding to whole sentences. If you set this to false, you can get examples for all phrases.

## Part (b) Questions

Answer the following questions in the answer and code cells below.

**Question 1.)** Implement Naive Bayes using `sklearn.naive_bayes.MultinomialNB`. Train on the training set and evaluate accuracy on the test set using `.predict(...)`. 

Your model should train almost instantly, and score 82.21% - not bad! On SST, this is actually a very strong baseline.

Recall that Naive Bayes can be interpreted as a linear model, where score is given by:

$$ \mathrm{score}(x) = \log \left(\frac{P(y = 1) \prod_{j=1}^n P(x_j | y = 1)}{P(y = 0) \prod_{j=1}^n P(x_j | y = 0)}\right) 
= \left( \log\hat{\theta}_1 - \log\hat{\theta}_0 \right) + \sum_{j=1}^n \left( \log\hat{\theta}_{1,j} - \log\hat{\theta}_{0,j} \right)$$

You can access the values $\log\hat{\theta}_{k,j}$ from the trained model using `nb.feature_log_prob_[k,j]`.

**Question 2.)** In the cell below, compute the weights $w_j = \left( \log\hat{\theta}_{1,j} - \log\hat{\theta}_{0,j} \right)$ of the linear model, and find the top 10 most negative and most positive weights. _(Hint: use `np.argsort` to get the indices of the most extreme elements.)_ Do the features you found make sense for this domain?

## Answer for Part (b).2
<a id="answers_b2"></a>

**2.)** Please see the cell outputs below. The most negative weights come from : "stupid", "suffers", "unfunny", "mess", "pointless", "flat", "poorly", "car", "tiresome", "disguise". The most positive weights come from : "powerful", "solid", "perfectly", "inventive", "refreshing", "touching", "riveting", "wonderful", "portrait", "heartwarming". Most of the words make sense. But it's peculiar the neutral term "car" is negative and the neutral term "portrait" is positive. Perhaps movies which are thoughtful tend to "portray" stories well; and "car" focused movies tend to be action-packed and unthoughtful. So their weights are purely associative with specific kinds of comments.

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#### YOUR CODE HERE ####
# Code for part (b).1
nb = MultinomialNB()
nb.fit(train_x_csr, train_y)
y_pred = nb.predict(test_x_csr)
#### END(YOUR CODE) ####
acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 82.21%


In [8]:
#### YOUR CODE HERE ####
# Code for part (b).2
linear_weights = np.squeeze(np.diff(nb.feature_log_prob_, axis = 0))  # populate this with actual values

top_negative_features = np.argsort(linear_weights)[:10]
top_positive_features = np.argsort(linear_weights)[-10:][::-1]

#### END(YOUR CODE) ####

print("Most negative features:")
for idx in top_negative_features:
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))
print("")
print("Most positive features:")
for idx in top_positive_features:
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))

Most negative features:
  stupid (-3.17)
  suffers (-3.08)
  unfunny (-2.97)
  mess (-2.79)
  pointless (-2.79)
  flat (-2.75)
  poorly (-2.72)
  car (-2.72)
  tiresome (-2.64)
  disguise (-2.56)

Most positive features:
  powerful (3.53)
  solid (3.45)
  perfectly (2.81)
  inventive (2.69)
  refreshing (2.63)
  touching (2.56)
  riveting (2.49)
  wonderful (2.49)
  portrait (2.41)
  heartwarming (2.41)


# Part (c): Examining Negation

While Naive Bayes performs well in the aggregate, as a linear model it's still limited in its ability to model complex phenomena in the data. Each feature - in this case, each word - contributes a weight to the total, and if the sum is $\ge 0$ we predict the example is positive. But what happens when we have an example with both positive and negative words? For instance:

```
[Brando 's performance fell short of the high standards set by his earlier work .]
[A thoughtful look at a painful incident that made headlines in 1995 .]
```

Run the cell below to evaluate your model on these examples. It should predict both as positive.

In [9]:
examples = ["Brando 's performance fell short of the high standards set by his earlier work .",
            "A thoughtful look at a painful incident that made headlines in 1995 ."]
id_lists = [ds.vocab.words_to_ids(ds.canonicalize(s.split())) for s in examples]
x = utils.id_lists_to_sparse_bow(id_lists, ds.vocab.size)
print(x.shape)
nb.predict(x)

(2, 16474)


array([1, 1], dtype=int32)

In [10]:
# first example weights
sorted(list(zip(map(lambda x: linear_weights[x],id_lists[0]), ds.vocab.ids_to_words(id_lists[0]),)), key=lambda x: x[0], reverse = True)

[(1.0991791851082366, 'performance'),
 (0.61367136932653565, 'standards'),
 (0.48013997670201292, 'work'),
 (0.1577996227186711, 'of'),
 (0.15123398046411562, 'his'),
 (0.035492833505731625, 'the'),
 (0.032134724769587564, "'s"),
 (0.0184617237532958, '.'),
 (-0.035609129127828432, 'by'),
 (-0.079475811233409743, '<unk>'),
 (-0.07947581123341152, 'high'),
 (-0.18483632689123652, 'set'),
 (-0.52130856351245036, 'short'),
 (-0.77262299179335514, 'fell'),
 (-0.99576654310756396, 'earlier')]

In [11]:
# second example weights
sorted(list(zip(map(lambda x: linear_weights[x],id_lists[1]), ds.vocab.ids_to_words(id_lists[1]),)), key=lambda x: x[0], reverse = True)

[(1.9129543534567972, 'thoughtful'),
 (1.3068185498864811, 'incident'),
 (0.82989447779617098, 'look'),
 (0.1113371160989729, 'a'),
 (0.1113371160989729, 'a'),
 (0.065706198611088062, 'at'),
 (0.054227506088519561, 'in'),
 (0.0184617237532958, '.'),
 (0.017253815225140912, 'that'),
 (-0.079475811233409743, '<unk>'),
 (-0.16355892844395115, 'made'),
 (-0.19725884688979356, 'painful'),
 (-0.24377886252468528, 'DGDGDGDG')]

## Part (c).1
<a id="answers_c1"></a>

**Question 1.)** Why does the model get the first example wrong? Why does it get the second one correct, despite the presence of the words "painful" and "incident"?

**1.)** In the first example, positive weights for "performance", "standards" and "work" outweights negative weights for "short" and "fell", steering the overall prediction to positive. Perhaps it is because "performance" is often used in "high performance", and "standards" is often used in "high standards". 

In the second example, positive weights for words "thoughtful" outweights negative weights of "painful". Perhaps "painful" can be used to describe "painful experience" of characters in great stories. "incident" actually gets a postive weight.

Please see weights from cell output from above.  

It's always important to look at individual examples, but let's try to do this a bit more systematically. Recall that SST gives us labels not only at the whole-sentence (root) level, but for individual phrases as well. We can use this to look for examples where polarity changes between different parts of the sentence. Here's one of the examples above:

In [158]:
ds.test_trees[210]

Tree('3', [Tree('4', [Tree('2', ['A']), Tree('4', ['thoughtful'])]), Tree('1', [Tree('1', [Tree('2', ['look']), Tree('2', [Tree('2', ['at']), Tree('2', [Tree('1', [Tree('2', ['a']), Tree('1', [Tree('1', ['painful']), Tree('2', ['incident'])])]), Tree('2', [Tree('2', ['that']), Tree('2', [Tree('3', [Tree('2', ['made']), Tree('2', ['headlines'])]), Tree('2', [Tree('2', ['in']), Tree('2', ['1995'])])])])])])]), Tree('2', ['.'])])])

The following cell will comb through the test set, looking for examples where there's some degree of ambiguity. We'll use a fairly crude heuristic for now: count up all the non-neutral phrases for a given sentence, and look at ones where there's a mix of both positive and negative labels.

In [12]:
df = ds.test

gb = df.groupby(by=['root_id'])
interesting_ids = []   # root ids, index into ds.test_trees
interesting_idxs = []  # DataFrame indices, index into ds.test
# This groups the DataFrame by sentence
for root_id, idxs in gb.groups.items():
    # Get the average score of all the phrases for this sentence
    mean = df.loc[idxs].label.mean()
    if (mean > 0.4 and mean < 0.6):
        interesting_ids.append(root_id)
        interesting_idxs.extend(idxs)
        
print("Found {:,} interesting examples".format(len(interesting_ids)))
# This will extract only the "interesting" sentences we found above
test_x_interesting, test_y_interesting = ds.as_sparse_bow("test", root_only=True, 
                                                          df_idxs=interesting_idxs)
print("Interesting ids (into ds.test_trees): ", interesting_ids)

Found 246 interesting examples
Interesting ids (into ds.test_trees):  [0, 27, 31, 32, 75, 80, 90, 96, 117, 124, 138, 140, 141, 160, 166, 186, 187, 205, 210, 212, 227, 232, 254, 269, 271, 285, 296, 307, 312, 327, 335, 373, 397, 399, 406, 407, 410, 426, 447, 511, 512, 516, 521, 534, 539, 563, 577, 588, 606, 610, 611, 637, 640, 645, 655, 662, 664, 713, 720, 721, 724, 739, 755, 758, 763, 776, 791, 793, 796, 802, 805, 810, 818, 840, 858, 887, 898, 899, 909, 910, 912, 929, 930, 961, 970, 973, 974, 975, 979, 1008, 1032, 1036, 1066, 1067, 1076, 1098, 1101, 1108, 1114, 1131, 1138, 1142, 1159, 1183, 1185, 1189, 1193, 1198, 1206, 1214, 1215, 1235, 1241, 1243, 1244, 1261, 1267, 1273, 1275, 1279, 1280, 1293, 1296, 1302, 1303, 1312, 1318, 1319, 1321, 1322, 1324, 1326, 1328, 1338, 1341, 1346, 1359, 1363, 1371, 1383, 1398, 1402, 1413, 1443, 1452, 1456, 1458, 1462, 1464, 1480, 1481, 1486, 1487, 1488, 1507, 1509, 1513, 1516, 1527, 1537, 1552, 1576, 1582, 1587, 1594, 1597, 1602, 1607, 1608, 1615, 1619, 1

## Part (c) continued

Answer the following in the cells below.

**Question 2.)** Examine a few of the "interesting" trees. What kinds of patterns do you see? What is the relation of the polarity of the sub-phrases to that of the whole sentence? Is this well-captured by a linear model?

**Question 3.)** Evaluate your model on `test_x_interesting` and compute accuracy. Does your model do better or worse on this slice of the data than on the rest of the test set? What is the _relative_ change in the error rate? _(For example, if you go from 90% accuracy to 85%, that's a 50% increase in error!)_

## Answers for Part (c).2 and 3
<a id="answers_c23"></a>

**2.)** Within these sentences, there tends to be a "twist". That is, explicit contrasts using words like "but", "although", "despite", "however", or implicit contrasts. The sentiment polarity of the sub-phrases switches and is depicted but the alternating red and blue branches below. The overall polarity of the sentence tend to consider its immediate children polarities and average between them. A linear model would consider the polarity of the sentence token by token, rather than decomposing by a subphrase structure. It will throw away information from subphrases which determine overall sentence polarity more directly.

**3.)** (On Piazza, instructors mention "Whoops! That should read entire test set, since we didn't ask you to calculate on {whole set - interesting}.") So In the following I am only comparing the interesting on the entire test set.

Accuracy on the interesting examples is 73.26% (error rate = 26.74%). Accuracy on the entire test set is 82.21% (error rate = 17.79%). The relative change in error rate is 50.309%.

In [27]:
#### YOUR CODE HERE ####
# Code for part (c).2
# Example: display(ds.test_trees[idx])
display(ds.test_trees[1952])
display(ds.test_trees[1929])
display(ds.test_trees[1755])
display(ds.test_trees[664])
#### END(YOUR CODE) ####

Tree('2', [Tree('3', [Tree('2', ['To']), Tree('3', [Tree('4', ['enjoy']), Tree('4', [Tree('3', [Tree('2', [Tree('2', ['this']), Tree('2', [Tree('2', ['movie']), Tree('2', ["'s"])])]), Tree('4', [Tree('3', ['sharp']), Tree('4', [Tree('2', ['dialogue']), Tree('4', [Tree('2', ['and']), Tree('4', [Tree('4', ['delightful']), Tree('2', ['performance'])])])])])]), Tree('2', [Tree('2', ['by']), Tree('2', [Tree('2', [Tree('2', ['Jolie']), Tree('2', ['and'])]), Tree('2', ['Burns'])])])])])]), Tree('0', [Tree('2', [',']), Tree('1', [Tree('2', ['you']), Tree('1', [Tree('1', [Tree('2', ['have']), Tree('1', [Tree('2', ['to']), Tree('2', [Tree('2', ['gloss']), Tree('1', [Tree('2', ['over']), Tree('1', [Tree('2', ['the']), Tree('2', [Tree('1', ['no']), Tree('2', [Tree('2', ['sense']), Tree('2', ['ending'])])])])])])])]), Tree('2', ['.'])])])])])

Tree('2', [Tree('3', [Tree('2', ['Despite']), Tree('3', [Tree('3', [Tree('3', [Tree('2', ['its']), Tree('3', [Tree('3', ['good']), Tree('2', ['nature'])])]), Tree('2', ['and'])]), Tree('3', [Tree('2', ['some']), Tree('3', [Tree('3', [Tree('3', ['genuinely']), Tree('3', ['funny'])]), Tree('2', ['moments'])])])])]), Tree('1', [Tree('2', [',']), Tree('0', [Tree('2', [Tree('3', ['Super']), Tree('2', ['Troopers'])]), Tree('1', [Tree('1', [Tree('2', ['suffers']), Tree('1', [Tree('2', ['from']), Tree('1', [Tree('2', [Tree('2', ['a']), Tree('1', [Tree('0', ['bad']), Tree('2', ['case'])])]), Tree('2', [Tree('2', ['of']), Tree('2', [Tree('1', ['arrested']), Tree('2', ['development'])])])])])]), Tree('2', ['.'])])])])])

Tree('2', [Tree('2', [Tree('2', ['The']), Tree('1', ['problem'])]), Tree('2', [Tree('2', [Tree('2', ['is']), Tree('2', [Tree('2', ['that']), Tree('2', [Tree('2', [Tree('2', ['rather']), Tree('2', [Tree('2', ['than']), Tree('2', [Tree('2', ['dramatizing']), Tree('2', [Tree('2', ['this']), Tree('2', ['premise'])])])])]), Tree('2', [Tree('2', [',']), Tree('2', [Tree('2', [Tree('2', ['Mr.']), Tree('2', ['Desplechin'])]), Tree('2', [Tree('2', ['is']), Tree('2', [Tree('3', ['content']), Tree('2', [Tree('2', ['to']), Tree('2', [Tree('2', ['state']), Tree('2', ['it'])])])])])])])])])]), Tree('2', ['.'])])])

Tree('3', [Tree('2', ['It']), Tree('3', [Tree('3', [Tree('2', ["'s"]), Tree('3', [Tree('4', [Tree('2', [Tree('2', ['an']), Tree('2', ['example'])]), Tree('3', [Tree('2', ['of']), Tree('3', [Tree('3', [Tree('3', ['sophisticated']), Tree('2', [Tree('2', [',']), Tree('2', ['challenging'])])]), Tree('2', ['filmmaking'])])])]), Tree('3', [Tree('2', ['that']), Tree('1', [Tree('2', [Tree('1', [Tree('3', [Tree('2', [Tree('2', ['stands']), Tree('2', [','])]), Tree('1', [Tree('2', ['despite']), Tree('0', [Tree('1', [Tree('2', ['its']), Tree('1', [Tree('2', ['noticeable']), Tree('2', ['lack'])])]), Tree('2', [Tree('2', ['of']), Tree('1', [Tree('2', ['emotional']), Tree('2', ['heft'])])])])])]), Tree('2', [','])]), Tree('2', [Tree('2', ['in']), Tree('3', [Tree('2', ['welcome']), Tree('2', ['contrast'])])])]), Tree('1', [Tree('2', ['to']), Tree('2', [Tree('1', [Tree('2', ['the']), Tree('1', [Tree('3', ['indulgent']), Tree('1', [Tree('1', ['dead-end']), Tree('2', ['experimentation'])])])]), Tree('3', [Tree('2', ['of']), Tree('2', [Tree('3', [Tree('2', ['the']), Tree('2', [Tree('2', ['director']), Tree('2', ["'s"])])]), Tree('2', [Tree('2', ['previous']), Tree('2', [Tree('3', ['Full']), Tree('2', ['Frontal'])])])])])])])])])])]), Tree('2', ['.'])])])

In [13]:
#### YOUR CODE HERE ####
# Code for part (c).3
# interesting
y_pred_interesting = nb.predict(test_x_interesting) 
acc = accuracy_score(test_y_interesting, y_pred_interesting)  # replace this with a real value for accuracy
#### END(YOUR CODE) ####
print("Accuracy on selected examples: {:.02f}%".format(100*acc))

Accuracy on selected examples: 73.26%


In [29]:
print ("relative change in error rate:",((1-73.26) - (1-82.21))/17.79)

relative change in error rate: 0.50309162450815
